In [ ]:
import os
import numpy as np
import tensorflow as tf


from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import Xception, InceptionResNetV2, MobileNetV3Large
from tensorflow.keras.optimizers import  Adam
from keras.models import Sequential, load_model
from keras.layers import Flatten, Dense, Dropout
from sklearn.metrics import precision_recall_fscore_support

#### **Base model preparing**

Load pre-trained base models with ImageNet weights. 

Choosing one of the three models to use or import more models.

In [ ]:
# if it has already been created and saved --> read the model
if os.path.isfile("xxx/saved_model/Xception_model.keras"):
    #load model
    base_model = load_model("xxx/saved_model/Xception_model.keras")

# if it has not already been created --> create the model and save it
else:
    base_model = Xception(
        include_top=False,  # exclude the original classification head
        weights="imagenet", # use pre-trained ImageNet weight
        input_shape=(224, 224, 3),
    )
    # save model
    if not os.path.exists("xxx/saved_model"):
        os.makedirs("xxx/saved_model")
    base_model.save("xxx/saved_model/Xception_model.keras")

In [ ]:
# if it has already been created and saved --> read the model
if os.path.isfile("xxx/saved_model/InceptionResNetV2_model.keras"):
    #load model
    base_model = load_model("xxx/saved_model/InceptionResNetV2_model.keras")

# if it has not already been created --> create the model and save it
else:
    base_model = InceptionResNetV2(
        include_top=False,
        weights="imagenet",
        input_shape=(224, 224, 3),
    )
    # save model
    if not os.path.exists("xxx/saved_model"):
        os.makedirs("xxx/saved_model")
    base_model.save("xxx/saved_model/InceptionResNetV2_model.keras")

In [ ]:
# if it has already been created and saved --> read the model
if os.path.isfile("xxx/saved_model/MobileNetV3_model.keras"):
    #load model
    base_model = load_model("xxx/saved_model/MobileNetV3_model.keras")

# if it has not already been created --> create the model and save it
else:
    base_model = MobileNetV3Large(
        include_top=False,
        weights="imagenet",
        input_shape=(224, 224, 3),
    )
    # save model
    if not os.path.exists("xxx/saved_model"):
        os.makedirs("xxx/saved_model")
    base_model.save("xxx/saved_model/MobileNetV3_model.keras")

In [ ]:
# Build model with fully connected layers
inputs = tf.keras.Input(shape=(224, 224, 3)) # define input data size
x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(1024, activation=("relu"), input_dim =512)(x)
x = Dense(512, activation=("relu"))(x)
x = Dense(256, activation=("relu"))(x)
x = Dropout(0.3)(x)
x = Dense(128, activation=("relu"))(x)
outputs = Dense(y_train.shape[1], activation=("softmax"))(x)
model = tf.keras.Model(inputs, outputs)

# Model summary
model.summary()

#### **Model pretraining**

Setup the initial hyperparameters.

Pre-train the base models with input data.

In [ ]:
""" Initialising the hyperparameters """

# Initialise no. of training samples for each batch
batch_size = 64

# No. of iterations
epochs = 50

# Learning rate
learn_rate = 0.001

# Using adam optimizer
adam = Adam( learning_rate=learn_rate)

# Compiling the model
model.compile(optimizer=adam, loss="categorical_crossentropy", metrics=["accuracy"])

# Learning Rate Annealer
train_lrr = ReduceLROnPlateau(monitor="accuracy", factor=0.2, patience=3, min_lr=1e-5)


In [ ]:
history = model.fit(
    x_train,
    y_train,
    epochs=epochs,
    steps_per_epoch=x_train.shape[0] // batch_size,
    # use validation dataset if needed
    #validation_data=(x_val, y_val),
    #validation_data=val_generator.flow(x_val, y_val, batch_size= batch_size),
    #validation_steps=250,
    callbacks=[train_lrr],
    verbose=1,
)
history = history.history

# save model 
np.save("xxx/pretrained_model_history.npy", history)
model.save("xxx/pretrained_model.keras")

# Making the predictions for the training data
y_train_pred = np.argmax(model.predict(x_train), axis=1)
y_train_true = np.argmax(y_train, axis=1)

precision, recall, f1_score, _ = precision_recall_fscore_support(y_train_true, y_train_pred, average=None)

for i in range(len(precision)):
    print(f"Class {i} - Precision: {precision[i]}, Recall: {recall[i]}, F1-score: {f1_score[i]}")

In [ ]:
# Making the predictions for the testing data
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", round(score[0], 3))
print("Test accuracy:", round(score[1], 3))

#### **Reload pre-trained model**

Reload the input data with the target dataset and skip all the pre-training steps to start fine-tuning.

Reload the pre-trained base model and exclude its fully connected layers, and build a new classification head for the final model fine-tuning.

In [ ]:
# Load a empty base model structure
base_model = MobileNetV3Large(
    include_top=False,
    weights=None,
    input_shape=(224, 224, 3),
)
# If use the double fine-tuned model as a start
if os.path.isfile("xxx/pretrained_model.keras"):
    # load model
    model = load_model("xxx/pretrained_model.keras")

reuse_layers = ["MobilenetV3large"] # A sample for utilising base layers from MobileNet model, use model_summary to ensure the name of base model is correct

for layer_name in reuse_layers:
    base_model = model.get_layer(layer_name)

In [ ]:
# Freeze layers in the base model
for layer in base_model.layers[:400]: # choose layer number to stay freezed in the base model, leave other layers trainable
    layer.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(1024, activation=("relu"), input_dim =512)(x)
x = Dense(512, activation=("relu"))(x)
x = Dense(256, activation=("relu"))(x)
x = Dropout(0.3)(x)
x = Dense(128, activation=("relu"))(x)
outputs = Dense(y_train.shape[1], activation=("softmax"))(x)
final_model = tf.keras.Model(inputs, outputs)

# Model summary
final_model.summary()

#### **Fine-tuning pre-trained model**

Initilise the hyperparameters and fine-tune the pre-trained model.

In [ ]:
""" Initialising the hyperparameters """

# Initialise no. of training samples for each batch
batch_size = 64

# No. of iterations
epochs = 50

# Learning rate
learn_rate = 0.001

# Using adam optimizer
adam = Adam( learning_rate=learn_rate)

# Compiling the model
final_model.compile(optimizer=adam, loss="categorical_crossentropy", metrics=["accuracy"])

# Learning Rate Annealer
train_lrr = ReduceLROnPlateau(monitor="accuracy", factor=0.2, patience=3, min_lr=1e-5)

# Save the best model during training
bst = ModelCheckpoint(filepath='xxx/best_model.keras', monitor='loss', save_best_only=True, mode='max', verbose=1)

In [ ]:
history = final_model.fit(
    x_train,
    y_train,
    epochs=epochs,
    steps_per_epoch=x_train.shape[0] // batch_size,
    #validation_data=(x_val, y_val),
    #validation_data=val_generator.flow(x_val, y_val, batch_size= batch_size),
    #validation_steps=250,
    callbacks=[train_lrr, bst],
    verbose=1,
)
history = history.history

# save model
np.save("xxx/model_history.npy", history)
final_model.save("xxx/model.keras")

In [ ]:
# Provide accuracy, loss performance
score = final_model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", round(score[0], 3))
print("Test accuracy:", round(score[1], 3))

# Provide precision, recall and f1-score performance
y_test_pred = np.argmax(final_model.predict(x_test), axis=1)
y_test_true = np.argmax(y_test, axis=1)

precision, recall, f1_score, _ = precision_recall_fscore_support(y_test_true, y_test_pred, average=None)

for i in range(len(precision)):
    print(f"Class {i} - Precision: {precision[i]}, Recall: {recall[i]}, F1-score: {f1_score[i]}")

In [ ]:
#Plot the training accuracy flow
import matplotlib.pyplot as plt2
plt2.plot(history['accuracy'])
plt2.title('model accuracy')
plt2.ylabel('accuracy')
plt2.xlabel('epoch')
plt2.legend(['train'], loc='upper left')
plt2.show()